In [1]:
%load_ext autoreload
%autoreload 2

# DeepSeek on VKS documents

## Using PostgreSQL to retrieval VKS documents


Define global variables

In [5]:
import os

from langchain_community.llms import VLLMOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from modules import utils

from IPython.display import display, Markdown, Latex

In [11]:
ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700
COLLECTION_NAME = "my_docs"

Load environment variables

In [7]:
envs = utils.load_env_to_dict(ENV_FILE_PATH)

## Prepare PostgreSQL vectorstore

In [13]:
from langchain_postgres.vectorstores import PGVector
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [15]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)

In [16]:
vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=envs["POSTGRESQL_URI"],
    use_jsonb=True,
)

## Retrieval

In [39]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [40]:
res = vector_store.similarity_search("VKS là gì?")
utils.print_documents(res)

Amount of documents: 4
Document 0:
1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Pla

## Integrated with DeepSeek R1

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [42]:
# Connect to vLLM API
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
)

prompt = ChatPromptTemplate.from_template("""
Bạn là một trợ lí ảo nhằm trả lời các CÂU HỎI cho khách hàng của VNGCloud.
Câu trả lời của bạn phải có liên quan đến NGỮ CẢNH được cung cấp. Nếu không liên quan hãy yêu cầu cung cấp thêm thông tin.
Bạn PHẢI trả lời bằng TIẾNG VIỆT

CÂU HỎI: {query}

NGỮ CẢNH:
{context}

TRẢ LỜI:
""")

chain = prompt | llm

## Prepare the context

In [43]:
query = "VKS là gì?"
context = utils.from_documents_to_context(vector_store.similarity_search(query))

display(Markdown(context[:1000]))

- Document 1:
1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, gi

In [46]:
query = "VKS là gì?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, khách hàng vừa hỏi VKS là gì. Tôi cần xem qua các tài liệu cung cấp để给 một câu trả lời chính xác và đầy đủ.

Trong Document 1, phần 1.1 đã giải thích VKS là dịch vụ quản lý Kubernetes của VNGCloud, giúp triển khai và quản lý ứng dụng container dễ dàng. Ngoài ra, VKS còn hỗ trợ các tính năng như tự động quản lý Control Plane, cập nhật phiên bản Kubernetes mới, tích hợp mạng lưới và lưu trữ, cũng như bànẹp bảo mật cao.

Document 2 mô tả về hai loại cluster và node group trong VKS, cùng với so sánh chi tiết giữa public và private cluster. Điều này cho thấy VKS rất linh hoạt và có thể đáp ứng các nhu cầu khác nhau về bảo mật và管理.

Document 3 và 4 đi sâu về việc cấu hình và sử dụng Network Load Balancer (NLB) trên VKS, hướng dẫn các bước cài đặt và các annotation cần thiết. Điều này chứng tỏ VKS hỗ trợ mạnh mẽ việc exposes dịch vụ và quản lý traffic.

Tapping vào đó, tôi cần tổng hợp lại các thông tin này để trả lời khách hàng. Câu trả lời phải bao gồm định nghĩa VKS

In [47]:
query = "Làm sao để expose service trong VKS?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, mình đang xem xét cách expose service trong VKS. Từ thông tin trong các Document, đặc biệt là Document 1 và Document 3, mình hiểu rằng VKS hỗ trợ tích hợp Load Balancer thông qua các annotations específico. Điều này cho phép người dùng dễ dàng expose các dịch vụ ra ngoài internet hoặc mạng nội bộ.

Trong Document 1, có đề cập đến việc VKS tích hợp Load Balancer (NLB/ALB) thông qua các driver được tích hợp sẵn như VNGCloud Controller Manager và VNGCloud Ingress Controller. Điều này ngụ ý rằng để expose một service, người dùng cần cấu hình Service trong Kubernetes với các annotations tương ứng.

Document 3 cụ thể hơn, liệt kê các annotations cần thiết như vks.vngcloud.vn/scheme với giá trị internet-facing hoặc internal. Điều này quyết định xem Load Balancer được tiếp cận từ internet hay chỉ trong mạng nội bộ. Mặt khác, các annotations khác liên quan đến security groups, inbound CIDRs giúp kiểm soáttraffic và bảo mật.

Mình cũng nhớ đến rằng để expose service trong K